In [12]:
pip install open3d==0.10.0

In [13]:
## IMPORT LIBRARIES
import numpy as np
import time
import open3d
import pandas as pd
import matplotlib.pyplot as plt

## USE http://www.open3d.org/docs/release/tutorial/Basic/

## CHALLENGE 1 - OPEN A FILE OF YOUR CHOICE AND VISUALIZE THE POINT CLOUD
# The supported extension names are: pcd, ply, xyz, xyzrgb, xyzn, pts.
pcd = open3d.io.read_point_cloud("/Users/LENOVO/Downloads/point_cloud_course-master/point_cloud_course-master/test_files/KITTI/000003.pcd")
#print(np.asarray([pcd.points]))
open3d.visualization.draw_geometries([pcd])

In [14]:
print(np.asarray([pcd.points]))

[[[47.18700027  8.56000042  1.83099997]
  [46.62900162  8.53499985  1.81200004]
  [46.06299973  8.57999992  1.79400003]
  ...
  [ 3.76900005 -1.426      -1.76300001]
  [ 3.79500008 -1.42200005 -1.77400005]
  [ 3.80299997 -1.41199994 -1.77499998]]]


In [15]:
## CHALLENGE 2 - VOXEL GRID DOWNSAMPLING
print(f"Points before downsampling: {len(pcd.points)} ")
downpcd=pcd.voxel_down_sample(voxel_size=0.2)

print(f"Points after downsampling: {len(downpcd.points)}")# DOWNSAMPLING
open3d.visualization.draw_geometries([downpcd])

Points before downsampling: 121795 
Points after downsampling: 23839


In [16]:
## CHALLENGE 3 - SEGMENTATION
t1=time.time()
_, inliers = downpcd.segment_plane(distance_threshold=0.3,ransac_n=3,num_iterations=250)
inlier_cloud=downpcd.select_by_index(inliers)
outlier_cloud=downpcd.select_by_index(inliers,invert=True)
inlier_cloud.paint_uniform_color([1,0,0])
outlier_cloud.paint_uniform_color([0,0,1])
open3d.visualization.draw_geometries([inlier_cloud,outlier_cloud])
t2=time.time()
print(f"time_taken: {t2-t1}")

time_taken: 12.172147035598755


In [17]:
## CHALLENGE 4 - CLUSTERING USING DBSCAN
with open3d.utility.VerbosityContextManager(open3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(outlier_cloud.cluster_dbscan(eps=0.5, min_points=5, print_progress=True))

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
outlier_cloud.colors = open3d.utility.Vector3dVector(colors[:, :3])
open3d.visualization.draw_geometries([outlier_cloud])

[Open3D DEBUG] Precompute Neighbours
[Open3D DEBUG] Done Precompute Neighbours
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 71
point cloud has 71 clusters


In [21]:
## BONUS CHALLENGE - CLUSTERING USING KDTREE AND KNN INSTEAD
pcd_tree = open3d.geometry.KDTreeFlann(outlier_cloud)
print("Paint the 15th point red.")
outlier_cloud.colors[15] = [1, 0, 0]
print("Find its 20 nearest neighbors, paint blue.")
[k, idx, _] = pcd_tree.search_knn_vector_3d(outlier_cloud.points[15], 20)
np.asarray(outlier_cloud.colors)[idx[1:], :] = [0, 0, 1]
open3d.visualization.draw_geometries([outlier_cloud])

Paint the 15th point red.
Find its 20 nearest neighbors, paint blue.


In [22]:
## CHALLENGE 5 - BOUNDING BOXES IN 3D
obbs=[]
indexes=pd.Series(range(len(labels))).groupby(labels,sort=False).apply(list).tolist()

In [23]:
Max_Points=1000
Min_Points=20
clusters=[]
for i in range(0,len(indexes)):
    nb_pts=len(outlier_cloud.select_by_index(indexes[i]).points)
    if (nb_pts>Min_Points and nb_pts<Max_Points):
        sub_cloud=outlier_cloud.select_by_index(indexes[i])
        #clusters.append(sub_cloud)
        #print(np.asarray([sub_cloud.points]))
        obb=sub_cloud.get_axis_aligned_bounding_box()
        obb.color=(0,0,1)
        obbs.append(obb)
        
    

In [24]:
list_of_visuals=[]
list_of_visuals.append(outlier_cloud)
list_of_visuals.extend(obbs)
list_of_visuals.append(inlier_cloud)
open3d.visualization.draw_geometries(list_of_visuals)

In [25]:
print(len(list_of_visuals))

19


In [11]:
print(np.asarray([clusters]))

[]
